In [22]:
import pandas as pd
import numpy as np

In [23]:
X_test = pd.read_csv('X_test_GgyECq8.csv')
X_train = pd.read_csv('X_train_Wwou3IE.csv')
y_train = pd.read_csv('y_train_jJtXgMX.csv')
y_pred = pd.read_csv('y_random_pt8afo8.csv')

In [31]:
X_train

,DELIVERY_START,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price,delta
0,2022-01-01 02:00:00+01:00,49439.0,3386.0,11487.0,44118.0,3035.0,0.0,79.248348,0.000000,NaN,-12587.0
1,2022-01-01 03:00:00+01:00,46511.0,3386.0,11487.0,44118.0,3143.0,0.0,61.776532,0.000000,NaN,-15623.0
2,2022-01-01 04:00:00+01:00,45158.0,3386.0,11487.0,44118.0,3288.0,0.0,44.291112,0.000000,NaN,-17121.0
3,2022-01-01 05:00:00+01:00,44779.0,3386.0,11487.0,44118.0,3447.0,0.0,36.127588,0.000000,NaN,-17659.0
4,2022-01-01 06:00:00+01:00,45284.0,3386.0,11487.0,44118.0,3679.0,0.0,30.983023,0.000000,NaN,-17386.0
...,...,...,...,...,...,...,...,...,...,...,...
10600,2023-03-29 19:00:00+02:00,50814.0,3386.0,11952.0,38320.0,7552.0,651.0,247.408490,7.821622,108.11,-11047.0
10601,2023-03-29 20:00:00+02:00,50628.0,3386.0,11952.0,38320.0,8338.0,109.0,155.795012,2.534054,125.66,-11477.0
10602,2023-03-29 21:00:00+02:00,48201.0,3386.0,11952.0,38320.0,9115.0,0.0,126.884684,0.000000,138.01,-14572.0
10603,2023-03-29 22:00:00+02:00,47967.0,3386.0,11952.0,38320.0,9636.0,0.0,156.669189,0.000000,136.74,-15327.0


In [32]:
y_train

,DELIVERY_START,spot_id_delta
0,2022-01-01 02:00:00+01:00,-36.874770
1,2022-01-01 03:00:00+01:00,-12.643588
2,2022-01-01 04:00:00+01:00,-1.950193
3,2022-01-01 05:00:00+01:00,1.938272
4,2022-01-01 06:00:00+01:00,0.199907
...,...,...
10600,2023-03-29 19:00:00+02:00,6.029303
10601,2023-03-29 20:00:00+02:00,13.576177
10602,2023-03-29 21:00:00+02:00,17.478945
10603,2023-03-29 22:00:00+02:00,17.559407


In [26]:
def preprocessing(df):
    df['delta'] = df['load_forecast'] - df['coal_power_available'] - df['gas_power_available'] - df['nucelear_power_available'] - df['wind_power_forecasts_average'] - df['solar_power_forecasts_average']

In [27]:
def compute_weighted_accuracy(y_actual, y_pred):
    actual = y_actual["spot_id_delta"].values
    predicted = y_pred["spot_id_delta"].values

    correct_direction = (np.sign(actual) == np.sign(predicted)).astype(int)

    weights = np.abs(actual)

    weighted_accuracy = np.sum(correct_direction * weights) / np.sum(weights)

    return round(weighted_accuracy, 2)

In [28]:
def launch(X, train=False):
    y = y_pred if not train else y_train.copy()
    preprocessing(X)

    # 1 benchmark: go long
    # y['spot_id_delta'] = 1

    # 2 delta load: load forecast - available generation
    y['spot_id_delta'] = np.where(X['delta'] < -17500, -1, 1)

    print(f"Shorts: {round(len(y[y['spot_id_delta']<0])/len(y)*100, 2)} %")
    if train:
        print(f"WA: {compute_weighted_accuracy(y_train, y)}")
    
    else:
        y.to_csv('y_pred.csv', index=False)

In [29]:
launch(X_train, train=True)

Shorts: 4.54 %
WA: 0.53


In [30]:
launch(X_test)

Shorts: 35.67 %
